<a href="https://colab.research.google.com/github/Vineet2107/EEG_Pipeline/blob/main/mainfile_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib
import pathlib

import mne
print(mne.__version__)

0.23.4


In [1]:
pip install mne

     |████████████████████████████████| 6.9 MB 9.3 MB/s 


In [4]:
#The files we will be loading
#files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2', '15_1', '15_2', '16_1', '16_2', '21_1', '21_2', '22_1', '22_2', '23_1', '23_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2', '33_1', '33_2', '34_1', '34_2', '35_1', '35_2', '46_1', '46_2', '47_1', '47_2', '48_1', '48_2', '49_1', '49_2', '50_1', '50_2', '69_1', '69_2', '70_1', '70_2', '71_1', '71_2', '72_1', '72_2']
files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2']

print(len(files))

12


In [ ]:
#Paths Arush
path = {}
for f in files:
    path[f] = 'data/other subjects set files/sub-AB{}_eeg_sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
    
path_channels_tsv = 'data/other subjects set files/sub-AB10_eeg_sub-AB10_task-gonogo_run-1_channels.tsv'

In [ ]:
#Loading raw data
unclean_raw = {}
for f in files:
    unclean_raw[f] = mne.io.read_raw_eeglab(path[f], eog=(), preload=True, uint16_codec=None, verbose=None)

In [ ]:

#plotting raw data
# rawab6_1.plot();
# rawab6_2.plot();
unclean_raw['10_1'].plot();
# rawab10_2.plot();
unclean_raw['12_1'].plot();
# rawab12_2.plot();
# rawab13_1.plot();
# rawab13_2.plot();
# rawab28_1.plot();
# rawab28_2.plot();
# rawab31_1.plot();
# rawab31_2.plot();
# rawab32_1.plot();
# rawab32_2.plot();

In [ ]:
#get type of channels from tsv file
import pandas as pd
channel_types_tsv = pd.read_csv(path_channels_tsv, delimiter = '\t')

def get_channel_types_dic(channel_types_tsv):
    channel_types = {}
    for i in range(channel_types_tsv.shape[0]):
        channel_types[channel_types_tsv['name'][i]] = channel_types_tsv['type'][i].lower()
    reassign_dic = {'HEO':'eog', 'VEO':'eog', 'R-Dia-X-(mm)':'misc', 'R-Dia-Y-(mm)':'misc'}
    for k in list(reassign_dic.keys()):
        if k in list(channel_types):
            channel_types[k] = reassign_dic[k]
    return channel_types

channel_types = get_channel_types_dic(channel_types_tsv)
channel_types

In [ ]:
#assigning channel types
for f in files:
    unclean_raw[f].set_channel_types(channel_types)

In [ ]:
#filtering the signals
for f in files:
    unclean_raw[f].filter(0.5, 60)

In [ ]:
unclean_raw['12_1'].plot_sensors(ch_type = 'eeg', sphere = 10);
# # rawab12_2.plot_sensors(ch_type = 'eeg', sphere = 10);

In [ ]:
unclean_raw['12_1'].plot_sensors(ch_type = 'eeg', kind = '3d');
# # rawab12_2.plot_sensors(ch_type = 'eeg', kind = '3d');

In [ ]:
unclean_raw['10_1']

In [ ]:
#Creating copy
raw = {}
for f in files:
    raw[f] = unclean_raw[f].copy()

In [ ]:
#Creating ICA objects
ica = {}
for f in files:
    ica[f] = mne.preprocessing.ICA(n_components = 20, random_state = 42)

In [ ]:
#Fitting ICA objects
for f in files:
    ica[f].fit(raw[f])

In [ ]:
#plotting sources and components
ica['10_1'].plot_sources(raw['10_1'])
ica['10_2'].plot_components(outlines = 'head', sphere = 10, ch_type = 'eeg')

In [ ]:
#To find and mark bad components
def mark_artifacts(ica, raw):
    ica.exclude = []
    ica.detect_artifacts(raw)
    eeg_bads = list(ica.exclude)
    ecg_bads = ica.find_bads_ecg(raw)[0]
    eog_bads = ica.find_bads_eog(raw)[0]
    ica.exclude = list(set(eeg_bads+ecg_bads+eog_bads))


In [ ]:
#Marking bad components
for f in files:
    mark_artifacts(ica[f], raw[f])

In [ ]:
ica['10_1'].plot_sources(raw['10_1'])

In [ ]:
#Applying ica
for f in files:
    ica[f].apply(raw[f], exclude = ica[f].exclude)

In [ ]:
events = {}
for f in files:
    events[f] = mne.events_from_annotations(raw[f])

In [ ]:
#Creating event id Dictionary
event_id = {
    "taskstart" : '9',
    "cue" : "1",
    "go" : "2",
    "button press" : "5",
    "no-go" : "4",
    "task end": "10",
    "error 1" : "3",
    "error 2" : "6",
    "error 3" : "7",
    "error 4" : "8",
    "error 5" : "11"
}
event_id

In [ ]:
#Creating epochs
epochs = {}
for f in files:
    epochs[f] = mne.Epochs(raw[f],
                   events = events[f][0],
                   event_id = events[f][1],)

In [ ]:
epochs['10_1'].plot();

In [ ]:
epochs['10_1'].get_data(picks = 'eeg').shape

In [ ]:
#to fix event ids
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]
def fix_event_ids(epochs, events):
    for i in range(epochs.events.shape[0]):
        epochs.events[i][2] = int(get_keys_from_value(events[1], epochs.events[i][2])[0])

In [ ]:
#fix event ids
for f in files:
    fix_event_ids(epochs[f], events[f])

In [ ]:
#Loading important libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#Getting epoch data
datadic = {}
for f in files:
    datadic[f] = epochs[f].get_data(picks = 'eeg')


In [ ]:
#Concatenating data
data = np.concatenate(list(datadic.values()), 
                      axis = 0)
print(data.shape)

#Changing the shape of data from (events, channel, time points) to (events, time points, channel)
datars = np.zeros((data.shape[0], data.shape[2], data.shape[1]))
for i in range(datars.shape[0]):
    datars[i] = np.transpose(data[i])
    
dims_lstm_1 = datars.shape[1]
dims_lstm_2 = datars.shape[2]
print(dims_lstm_1, "X", dims_lstm_2)
datars.shape

In [ ]:
n_trials = data.shape[0]


data = data.reshape(n_trials, -1)
print(data.shape)
dims_ip = data.shape[1]
dims_ip


In [ ]:
#Getting y
ydic = {}
for f in files:
    ydic[f] = epochs[f].events[:, 2]
y = np.concatenate(list(ydic.values()), axis = 0)

In [ ]:
#Preparing main dataset
y = y.reshape(-1, 1)
# y = y[...,np.newaxis]
datarscopy = datars.copy()
datarscopy = datarscopy.reshape(datars.shape[0], -1)

print(datars.shape)
print(y.shape)
main_dataset = np.concatenate((datarscopy, y), axis = 1)
print(main_dataset.shape)

#Defining main x and y for training
main_x = main_dataset[:, :-1]
main_y = main_dataset[:, -1]
print(main_x.shape)
print(main_y.shape)


In [ ]:
# Clearing memory
del unclean_raw
del raw
del ica
del events
del epochs


In [ ]:
#Removing events which are not needed for current task
dataset = main_dataset.copy()
print(dataset.shape)
dataset = dataset[dataset[:, -1]!=9]
dataset = dataset[dataset[:, -1]!=10]

dataset = dataset[dataset[:, -1]!=3]
dataset = dataset[dataset[:, -1]!=6]
dataset = dataset[dataset[:, -1]!=7]
dataset = dataset[dataset[:, -1]!=8]
dataset = dataset[dataset[:, -1]!=11]

print(dataset.shape)

#Defining x and y
x = dataset[:, :-1]
y = dataset[:, -1]
print(x.shape)
print(y.shape)
print(list(set(y)))

permutations = np.random.permutation(len(x))
# print(permutation)
print(x[-1])
x = x[permutations]
y = y[permutations]
print(x[-1])

xmlp = x.copy()
ymlp = y.copy()

xlstm = x.copy().reshape((-1, dims_lstm_1, dims_lstm_2))
ylstm = y.copy()

xcnn = x.copy().reshape((-1, dims_lstm_1, dims_lstm_2))
ycnn = y.copy()

In [ ]:
#Encoding y
dict = {1.0: 0,2.0: 1, 4.0: 2, 5.0: 3}

for i in range(len(y)):
    y[i] = dict[y[i]]
    ylstm[i] = dict[ylstm[i]]

print(list(set(y)))

In [ ]:
num_classes = 4
y = tf.keras.utils.to_categorical(y, num_classes)
ymlp = y.copy()
ylstm = y.copy()
ycnn = y.copy()
# yab10_1[0]
print(y[:5])

In [ ]:
train_test_split = int(0.8 * len(xmlp))
print(train_test_split)

x_train_mlp = xmlp[:train_test_split]
y_train_mlp = ymlp[:train_test_split]

x_test_mlp = xmlp[train_test_split:]
y_test_mlp = ymlp[train_test_split:]

# x_mlp_train = xmlp[:]

In [ ]:
def get_cyclic_generator(features, labels, batch_size = 1):
  while True:
    for n in range(features.shape[0]//batch_size):
      yield (features[n*batch_size: (n+1)*batch_size], labels[n*batch_size: (n+1)*batch_size])
    permuted = np.random.permutation(len(features))
    features = features[permuted]
    labels = labels[permuted]

In [ ]:
train_batch_size = 32
train_cyclic_generator_mlp = get_cyclic_generator(x_train_mlp, y_train_mlp, batch_size = train_batch_size)
# test_cyclic_generator_mlp = get_cyclic_generator(x_test_mlp, y_test_mlp, batch_size = train_batch_size)


In [ ]:
#Creating model architecture
inp_shape = (dims_ip, )
ip = tf.keras.Input(shape = inp_shape)
dense1 = tf.keras.layers.Dense(units = 32, activation = 'relu', kernel_initializer = 'random_normal')(ip)
drop = tf.keras.layers.Dropout(.4)(dense1)
dense2 = tf.keras.layers.Dense(units = 16, activation = 'relu', kernel_initializer = 'random_normal')(drop)
drop2 = tf.keras.layers.Dropout(.4)(dense2)
out = tf.keras.layers.Dense(num_classes, activation = 'softmax')(drop2)

mlp = tf.keras.Model(inputs = ip, outputs = out)
mlp.summary()
# inp_shape = (None, )
# review_sequence = tf.keras.Input(shape = inp_shape)
# embedding_sequence = tf.keras.layers.Embedding(input_dim=100+1, output_dim=32, input_shape=(review_sequence.shape), mask_zero=False)(review_sequence)
# average_embedding = tf.keras.layers.GlobalAveragePooling1D()(embedding_sequence)
# positive_probability = tf.keras.layers.Dense(units=1, activation='sigmoid')(average_embedding)

# model = tf.keras.Model(inputs = review_sequence, outputs = positive_probability)

In [ ]:
#Creating model architecture
inp_shape = (dims_ip, )
ip = tf.keras.Input(shape = inp_shape)
dense1 = tf.keras.layers.Dense(units = 32, activation = 'relu', kernel_initializer = 'random_normal')(ip)
drop = tf.keras.layers.Dropout(.4)(dense1)
dense2 = tf.keras.layers.Dense(units = 16, activation = 'relu', kernel_initializer = 'random_normal')(drop)
drop2 = tf.keras.layers.Dropout(.4)(dense2)
out = tf.keras.layers.Dense(num_classes, activation = 'softmax')(drop2)

mlp = tf.keras.Model(inputs = ip, outputs = out)
mlp.summary()
# inp_shape = (None, )
# review_sequence = tf.keras.Input(shape = inp_shape)
# embedding_sequence = tf.keras.layers.Embedding(input_dim=100+1, output_dim=32, input_shape=(review_sequence.shape), mask_zero=False)(review_sequence)
# average_embedding = tf.keras.layers.GlobalAveragePooling1D()(embedding_sequence)
# positive_probability = tf.keras.layers.Dense(units=1, activation='sigmoid')(average_embedding)

# model = tf.keras.Model(inputs = review_sequence, outputs = positive_probability)

In [ ]:
mlp_history = mlp.fit(train_cyclic_generator_mlp,
#                         validation_data = test_cyclic_generator_mlp,
                      validation_data = (x_test_mlp, y_test_mlp),
                      steps_per_epoch= 449,
                      epochs = 50,
                      callbacks = [mlp_earlystop, mlp_checkpoint]) #Fitting the model


In [ ]:
#Plotting model history
plt.plot(mlp_history.history['acc'])
plt.plot(mlp_history.history['loss'])
plt.legend(['Training Accuracy', 'Training Loss'])
plt.title('Training History')
plt.ylabel('Value')
plt.xlabel('epochs')
plt.show()
print("Initial training accuracy is: ", mlp_history.history['acc'][0])
print("Final training accuracy is: ", mlp_history.history['acc'][-1])

print("Initial training loss is: ", mlp_history.history['loss'][0])
print("Final training loss is: ", mlp_history.history['loss'][-1])

In [ ]:
#Plotting model history
plt.plot(mlp_history.history['val_acc'])
plt.plot(mlp_history.history['val_loss'])
plt.legend(['Valdation Accuracy', 'Validation Loss'])
plt.title('Validation History')
plt.ylabel('Value')
plt.xlabel('epochs')
plt.show()
print("Initial validation accuracy is: ", mlp_history.history['val_acc'][0])
print("Final validation accuracy is: ", mlp_history.history['val_acc'][-1])

print("Initial validation loss is: ", mlp_history.history['val_loss'][0])
print("Final validation loss is: ", mlp_history.history['val_loss'][-1])


In [ ]:
train_test_split = int(0.8 * len(xlstm))
print(train_test_split)

x_train_lstm = xlstm[:train_test_split]
y_train_lstm = ylstm[:train_test_split]

x_test_lstm = xlstm[train_test_split:]
y_test_lstm = ylstm[train_test_split:]


In [ ]:
def get_cyclic_generator(features, labels, batch_size = 1):
  while True:
    for n in range(features.shape[0]//batch_size):
      yield (features[n*batch_size: (n+1)*batch_size], labels[n*batch_size: (n+1)*batch_size])
    permuted = np.random.permutation(len(features))
    features = features[permuted]
    labels = labels[permuted]

In [ ]:
train_batch_size = 32
train_cyclic_generator_lstm = get_cyclic_generator(x_train_lstm, y_train_lstm, batch_size = train_batch_size)
# test_cyclic_generator = get_cyclic_generator(x_test, y_test, batch_size = train_batch_size)

In [ ]:
#Creating model architecture
inp_shape = (dims_lstm_1, dims_lstm_2)
ip = tf.keras.Input(shape = inp_shape)
lstm = tf.keras.layers.LSTM(32, return_sequences=True)(ip)
# drop = tf.keras.layers.Dropout(.4)(lstm)
# lstm = tf.keras.layers.LSTM(16, return_sequences=True)(ip)
# drop = tf.keras.layers.Dropout(.4)(lstm)
flatten = tf.keras.layers.Flatten()(lstm)
dense1 = tf.keras.layers.Dense(units = 32, kernel_initializer = 'random_normal')(flatten)
leakyRelu = tf.keras.layers.LeakyReLU()(dense1)
drop1 = tf.keras.layers.Dropout(.4)(leakyRelu)
dense2 = tf.keras.layers.Dense(units = 16, kernel_initializer = 'random_normal')(drop1)
leakyRelu2 = tf.keras.layers.LeakyReLU()(dense2)
drop2 = tf.keras.layers.Dropout(.4)(dense2)
out = tf.keras.layers.Dense(num_classes, activation = 'softmax')(drop2)

lstm = tf.keras.Model(inputs = ip, outputs = out)
lstm.summary()
# inp_shape = (None, )
# review_sequence = tf.keras.Input(shape = inp_shape)
# embedding_sequence = tf.keras.layers.Embedding(input_dim=100+1, output_dim=32, input_shape=(review_sequence.shape), mask_zero=False)(review_sequence)
# average_embedding = tf.keras.layers.GlobalAveragePooling1D()(embedding_sequence)
# positive_probability = tf.keras.layers.Dense(units=1, activation='sigmoid')(average_embedding)

# model = tf.keras.Model(inputs = review_sequence, outputs = positive_probability)
